# AUTOMATIC1111's Stable Diffusion WebUI

https://github.com/AUTOMATIC1111/stable-diffusion-webui

Loosely based on https://colab.research.google.com/drive/1kw3egmSn-KgWsikYvOMjJkVDsPLjEMzl

**Guides**
- [Getting started on Paperspace](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/Docs/Paperspace%20Guide%20for%20Retards.md)
- [Using the WebUI](https://rentry.org/voldy)
- [Using the Inpainter](https://rentry.org/drfar)
- [Textual Inversion](https://rentry.org/aikgx)
- [Crowd-Sourced Prompts](https://lexica.art/)
- [Artist Name Prompts](https://sgreens.notion.site/sgreens/4ca6f4e229e24da6845b6d49e6b08ae7?v=fdf861d1c65d456e98904fe3f3670bd3)
- [Stable Diffusion Models](https://cyberes.github.io/stable-diffusion-models)
- [Textual Inversion Models](https://cyberes.github.io/stable-diffusion-textual-inversion-models/)
- [Have I Been Trained?](https://haveibeentrained.com/)

# Installation and Setup

You must reinstall everything each time you restart the machine. If already downloaded, dependencies will be auto-updated.

**Where to store the models**

`/storage/` is persistent storage shared across all machines on your account. Mounted to your machine.

`/notebooks/` is storage for this notebook only. This directory has to be copied into your machine which can increase start/stop times if the directory is very large. To avoid this, put large files in `/storage/`.

`/tmp/` <mark style="background-color:lime">is not a persistent directory, meaning your files there will be deleted when the machine turns off.</mark>

<br>

<mark style="background-color: #ff780082">If you are having storage issues</mark>, set `repo_storage_dir` to `/tmp/stable-diffusion`.

<br><br>

<mark>You must uncomment the correct section and run the block below or else the notebook won't work!</mark>

Select the section you want and do `ctrl + /` to uncomment.

Make sure there isn't a space in front of any variable. Don't put a trailing slash on directory paths. If you change any settings here, rerun this cell.

In [ ]:
# Free tier
# symlink_to_notebooks = True                          # Enables the creation of symlinks back to /notebooks/
# model_storage_dir = '/tmp/stable-diffusion/models'   # Where the models will be downloaded to.
# repo_storage_dir = '/notebooks'                      # Where the repository will be downloaded to.
# pip_cache_dir = None                                 # Cache pip downloads to this directory so you don't have to re-download everything.

# Paid Tier
# symlink_to_notebooks = False
# model_storage_dir = '/storage/models'
# repo_storage_dir = '/notebooks'
# pip_cache_dir = '/storage/pip/cache'


# Other optional settings
# You don't have to change these if you don't want to

activate_xformers = False                              # Enables the xformers optimizations using pre-built wheels.
                                                       # Setting to True will automatically set up your environment/machine for xformers. 

link_novelai_anime_vae = False                         # Enables the linking of animevae.pt to each of the NovelAI models.
                                                       # Set to True if you've downloaded both the NovelAI models and hypernetworks.

download_scripts = False                               # Download custom scripts? Only reason why you would leave it disabled is because it may
                                                       # take a while to complete.

activate_deepdanbooru = False                          # Enable and install DeepDanbooru -> https://github.com/KichangKim/DeepDanbooru

activate_medvram = True                                # Enable medvram option.
                                                       # These are model optimizations which will reduce VRAM usage at the expense of some speed.
                                                       # Set to False if you have a lot of VRAM.

disable_pickle_check = False                           # Disable the automatic check for unexpected data in model files.
                                                       # Leave this set to False unless you have a reason to disable the check.

gradio_port = False                                    # Launch Gradio on a specific port. Set to False to let Gradio choose a port.
                                                       # This disables online Gradio app mode and you will only be able to access it on your local network.

gradio_auth = False                                    # Enable gradio_auth and insecure-extension-access option.
                                                       # Set to "me:password" to enable.

search_paperspace_datasets = True                      # Enable searching for checkpoints in /datasets to link to the webui

ui_theme = None                                        # Set the WEB UI theme. Values can be None (default) or 'dark'.

insecure_extension_access = False                      # Force enable extensions without a password.
                                                       # If you don't set a password, anyone can install and run arbitrary code on your machine!
                                                       # Instead, use gradio_auth which will automatically enable extensions when set.

export_storage_dir = '/notebooks/exports'              # Where the generated images will be exported to.
        
# ===================================================================================================
# Save variables to Jupiter's temp storage so we can access it even if the kernel restarts.
%store symlink_to_notebooks model_storage_dir repo_storage_dir export_storage_dir activate_xformers link_novelai_anime_vae download_scripts activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth search_paperspace_datasets ui_theme insecure_extension_access pip_cache_dir

<mark>If you see any errors please check your settings!</mark>

**Don't forget, there's a block in the Tools section at the bottom to update this notebook to [the latest version](https://github.com/Engineer-of-Stuff/stable-diffusion-paperspace/blob/main/StableDiffusionUI_Voldemort_paperspace.ipynb) on GitHub.**

## Clone the WebUI repository

In [ ]:
import os
# You'll see this little code block at the beginning of every cell.
# It makes sure you have ran the first block that defines your settings.
try:
    %store -r symlink_to_notebooks model_storage_dir repo_storage_dir
    test = [symlink_to_notebooks, model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

%cd /notebooks/

def delete_broken_symlinks(path):
    # make sure to pass this function a path without a trailing slash
    for file in os.listdir(path):
        if os.path.islink(f'{path}/{file}') and not os.path.exists(os.readlink(f'{path}/{file}')):
            print(f'Symlink broken, removing: {file}')
            os.unlink(f'{path}/{file}')

def update_repo_if_not_exists(path, repo_clone_url, pre=None):
    if pre is not None:
        pre()    
    if not os.path.exists(path):
        !git clone "{repo_clone_url}" "{path}"
    else:
        print(f'{repo_clone_url.split("/")[-1]} already downloaded, updating...')
        !cd "{path}" && git pull # no % so we don't interfere with the main process

def init_free():
    if (symlink_to_notebooks and repo_storage_dir != '/notebooks'):
        delete_broken_symlinks('/notebooks/') # remove broken symlinks since it might have been installed in a non-persistent directory
        if not os.path.exists(repo_storage_dir):
            !mkdir -p "{repo_storage_dir}"
            !ln -s "{repo_storage_dir}" /notebooks/
            !ls -la /notebooks/stable-diffusion
update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui', 'https://github.com/AUTOMATIC1111/stable-diffusion-webui', init_free)

## Install requirements and download repositories

In [ ]:
try:
    %store -r symlink_to_notebooks model_storage_dir repo_storage_dir activate_xformers download_scripts activate_deepdanbooru pip_cache_dir
    test = [symlink_to_notebooks, model_storage_dir, repo_storage_dir, activate_xformers, download_scripts, activate_deepdanbooru, pip_cache_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

%cd "{repo_storage_dir}/stable-diffusion-webui"

!pip install --upgrade pip
!pip install --upgrade wheel setuptools

if pip_cache_dir:
    !pip install git+https://github.com/pixelb/crudini.git
    !mkdir -p "{pip_cache_dir}"
    !python3 -m crudini --set /etc/pip.conf global cache-dir "{pip_cache_dir}"
    !echo "Set pip cache directory: $(pip cache dir)"

import os

# Import launch.py which will automatically run the install script but not launch the WebUI.
import launch
launch.prepare_environment()

# The installer isn't installing deepdanbooru right now so we'll do it manually.
if activate_deepdanbooru:
    !pip install "git+https://github.com/KichangKim/DeepDanbooru.git@edf73df4cdaeea2cf00e9ac08bd8a9026b7a7b26#egg=deepdanbooru[tensorflow]" # tensorflow==2.10.0 tensorflow-io==0.27.0 flatbuffers==1.12

# latent-diffusion is a requirement but launch.py isn't downloading it so we'll do it manually.
# TODO: can this be removed?
if not os.path.exists(f'{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion'):
    !git clone https://github.com/crowsonkb/k-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/k-diffusion"
    !git clone https://github.com/Hafiidz/latent-diffusion.git "{repo_storage_dir}/stable-diffusion-webui/repositories/latent-diffusion"

# For things in this notebook
!pip install requests
!pip install gdown

# Download popular custom scripts. This is basically remote code execution so be careful.
# See https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Custom-Scripts
if download_scripts:
    import shutil
    import requests
    !pip install moviepy==1.0.3
    !apt update
    !apt install -y potrace python3-tk

    def download_file_dir(url, output_dir):
        # output_dir must have a trailing slash
        local_filename = url.split('/')[-1]
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(f'{output_dir}{local_filename}', 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        return local_filename
    def do_script_download(scripts_list, domain, path):
        for item in scripts_list:
            download_file_dir(f'https://{domain}/{item}', path)
            print(f'{item.split("/")[-1]} downloaded...')

    do_script_download([
        'GRMrGecko/stable-diffusion-webui-automatic/advanced_matrix/scripts/advanced_prompt_matrix.py',
        'dfaker/stable-diffusion-webui-cv2-external-masking-script/main/external_masking.py',
        'memes-forever/Stable-diffusion-webui-video/main/videos.py',
        'yownas/seed_travel/main/scripts/seed_travel.py',
        'Animator-Anon/Animator/main/animation.py',
        'Filarius/stable-diffusion-webui/master/scripts/vid2vid.py',
        'GeorgLegato/Txt2Vectorgraphics/main/txt2vectorgfx.py',
        'yownas/shift-attention/main/scripts/shift_attention.py',
        'DiceOwl/StableDiffusionStuff/main/loopback_superimpose.py',
        'Engineer-of-Stuff/stable-diffusion-paperspace/main/lfs/save_steps.py',
        'Pfaeff/sd-web-ui-scripts/main/moisaic.py'
    ], 'raw.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

    do_script_download([
        'dfaker/f88aa62e3a14b559fe4e5f6b345db664/raw/791dabfa0ab26399aa2635bcbc1cf6267aa4ffc2/alternate_sampler_noise_schedules.py',
        'camenduru/9ec5f8141db9902e375967e93250860f/raw/c1a03eb447548adbef1858c0e69d3567a390d2f4/run_n_times.py'
    ], 'gist.githubusercontent.com', f'{repo_storage_dir}/stable-diffusion-webui/scripts/')

    # Download and set up txt2img2img
    update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root', 'https://github.com/ThereforeGames/txt2img2img.git')
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/scripts" "{repo_storage_dir}/stable-diffusion-webui"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/txt2img2img" "{repo_storage_dir}/stable-diffusion-webui"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2img2img_root/venv" "{repo_storage_dir}/stable-diffusion-webui"

    # Download and set up txt2mask
    update_repo_if_not_exists(f'{repo_storage_dir}/stable-diffusion-webui/txt2mask', 'https://github.com/ThereforeGames/txt2mask.git')
    !echo "Copying txt2mask..."
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/repositories/clipseg" "{repo_storage_dir}/stable-diffusion-webui/repositories"
    !cp -r "{repo_storage_dir}/stable-diffusion-webui/txt2mask/scripts/" "{repo_storage_dir}/stable-diffusion-webui/"

    # Install the dynamic-prompts/wildcard script
    # !git clone https://github.com/adieyal/sd-dynamic-prompting/ extensions/dynamic-prompts

if activate_xformers:
    print('Installing xformers...')
    import subprocess
    def download_release(url):
        binary = 'xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl' # have to save the binary as a specific name that pip likes
        tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
        !wget "{url}" -O "{tmp_dir}/{binary}"
        return os.path.join(tmp_dir, binary)

    # Set up pip packages
    !pip uninstall -y torch  torchvision torchaudio # Remove existing pytorch install.
    !pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113 # Install pytorch for cuda 11.3
    s = subprocess.getoutput('nvidia-smi')
    if 'A4000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A4000-Oct-28-2022/a4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A5000-Nov-1-2022/a5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A6000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A6000-Nov-1-2022/a6000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'P5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/P5000-Nov-1-2022/p5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'RTX 4000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-4000-Nov-1-2022/rtx4000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'RTX 5000' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/RTX-5000-Nov-1-2022/rtx5000-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'A100' in s:
        xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    elif 'M4000' in s:
        print('xformers for M4000 hasn\'t been built yet.')
        # xformers_whl = download_release('https://github.com/Cyberes/xformers-compiled/releases/download/A100-Nov-1-2022/a100-xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl')
    else:
        print('GPU not matched to xformers binary so a one-size-fits-all binary was installed. If you have any issues, please build xformers using the Tools block below.')
        xformers_whl = download_release('https://raw.githubusercontent.com/Cyberes/xformers-compiled/main/various/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl')
    !pip install --force-reinstall "{xformers_whl}"

# Make sure your models storage directory exists
!mkdir -p "{model_storage_dir}/hypernetworks"
!mkdir -p "{model_storage_dir}/vae"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/models/hypernetworks"

# Link the output folders to /notebooks/outputs
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/log/images"
!mkdir -p "{repo_storage_dir}/stable-diffusion-webui/outputs"
!ln -s "{repo_storage_dir}/stable-diffusion-webui/outputs" /notebooks/
!ln -s "{repo_storage_dir}/stable-diffusion-webui/log" "{repo_storage_dir}/stable-diffusion-webui/outputs"

# Link /storage/ to /notebooks/
!ln -s /storage/ /notebooks/

!echo -e "\n===================================\nDone! If you're seeing this the process has exited successfully.\n"

## Download the Model

You don't need to repeat this step if you've already downloaded the models.

<br>

**There are additional models available here: https://cyberes.github.io/stable-diffusion-models**

Textual inversion: https://cyberes.github.io/stable-diffusion-textual-inversion-models

DreamBooth: https://cyberes.github.io/stable-diffusion-dreambooth-library

<br>

### Filesize and Storage Disclaimer

Paperspace free tier has only 5GB of storage space. If you're having storage issues, here are a few suggestions.
1. Download everything to `/tmp/` each time you start the machine.
2. Add a payment method to your account. Storage overages are billed at \$0.29/GB and billing occurs monthly and runs at midnight on the first of each month. With a payment method on file, Paperspace will let you use more storage and if you time it right you shouldn't actually be charged for it.
3. Upgrade to a Pro account. They'll give you 15GB and you'll get longer runtimes and more powerful free GPUs.
4. Use my referral code `KQLRH37`. You'll get \$10 credit that you should be able to put towards the storage overage charges. Redeem the code at the bottom of the Billing page.

### Torrent Instructions

Aria2 may show some errors/warnings while downloading. Those are fine, when it eventually says "Download Complete" that means everything worked as it should.

### Stable Diffusion 2.0

**768x768**

This model can generate images 768 pixels by 768 pixels.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/stabilityai/stable-diffusion-2/resolve/main/768-v-ema.ckpt -d "{model_storage_dir}" -o "sd-v2-0-768-v-ema.ckpt"
!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O "{model_storage_dir}/sd-v2-0-768-v-ema.yaml"

**512x512 Base**

Referred to as the "base" model, this model generates images in the standard 512x512 resolution.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/stabilityai/stable-diffusion-2-base/resolve/main/512-base-ema.ckpt -d "{model_storage_dir}" -o "sd-v2-0-512-base-ema.ckpt"
!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O "{model_storage_dir}/sd-v2-0-512-base-ema.yaml"

**Depth Model**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt -d "{model_storage_dir}" -o "sd-v2-0-512-depth-ema.ckpt"
!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O "{model_storage_dir}/sd-v2-0-512-depth-ema.yaml"

**4x Upscaler**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/stabilityai/stable-diffusion-x4-upscaler/resolve/main/x4-upscaler-ema.ckpt -d "{model_storage_dir}" -o "sd-v2-0-x4-upscaler-ema.ckpt"
!wget https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/x4-upscaling.yaml -O "{model_storage_dir}/sd-v2-0-x4-upscaler-ema.yaml"

### Stable Diffusion 1.0

**v1.5**

Paperspace includes this model in its public data sources, which don't use up your storage quota. To add it, click on `Data Sources` in the toolbar, `Public`, and `stable-diffusion-classic`. The file is mounted at `/datasets/stable-diffusion-classic/`. You only need to do this once, it will stay mounted between sessions. Make sure the setting `search_paperspace_datasets` is set to `True` so the program will link it to the WebUI.

Otherwise, you can download it yourself:

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:2daef5b5f63a16a9af9169a529b1a773fc452637&dn=v1-5-pruned-emaonly.ckpt&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2f9.rarbg.com%3a2810%2fannounce&tr=udp%3a%2f%2ftracker.openbittorrent.com%3a6969%2fannounce&tr=udp%3a%2f%2fopentracker.i2p.rocks%3a6969%2fannounce&tr=https%3a%2f%2fopentracker.i2p.rocks%3a443%2fannounce&tr=http%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2ftracker.torrent.eu.org%3a451%2fannounce&tr=udp%3a%2f%2fopen.stealth.si%3a80%2fannounce&tr=udp%3a%2f%2fvibe.sleepyinternetfun.xyz%3a1738%2fannounce&tr=udp%3a%2f%2ftracker2.dler.org%3a80%2fannounce&tr=udp%3a%2f%2ftracker1.bt.moack.co.kr%3a80%2fannounce&tr=udp%3a%2f%2ftracker.zemoj.com%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.tiny-vps.com%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.theoks.net%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.publictracker.xyz%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.monitorit4.me%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.moeking.me%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.lelux.fi%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.dler.org%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.army%3a6969%2fannounce"
!mv "v1-5-pruned-emaonly.ckpt" "sd-v1-5-pruned-emaonly.ckpt"

**v1.5 Inpainting**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:b523a9e71ae02e27b28007eca190f41999c2add1&dn=sd-v1-5-inpainting.ckpt&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2f9.rarbg.com%3a2810%2fannounce&tr=udp%3a%2f%2ftracker.openbittorrent.com%3a6969%2fannounce&tr=http%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2fopentracker.i2p.rocks%3a6969%2fannounce&tr=https%3a%2f%2fopentracker.i2p.rocks%3a443%2fannounce&tr=udp%3a%2f%2ftracker.torrent.eu.org%3a451%2fannounce&tr=udp%3a%2f%2fopen.stealth.si%3a80%2fannounce&tr=udp%3a%2f%2fvibe.sleepyinternetfun.xyz%3a1738%2fannounce&tr=udp%3a%2f%2ftracker2.dler.org%3a80%2fannounce&tr=udp%3a%2f%2ftracker1.bt.moack.co.kr%3a80%2fannounce&tr=udp%3a%2f%2ftracker.zemoj.com%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.tiny-vps.com%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.theoks.net%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.swateam.org.uk%3a2710%2fannounce&tr=udp%3a%2f%2ftracker.publictracker.xyz%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.pomf.se%3a80%2fannounce&tr=udp%3a%2f%2ftracker.monitorit4.me%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.moeking.me%3a6969%2fannounce&tr=udp%3a%2f%2ftracker.lelux.fi%3a6969%2fannounce"

**v1.4**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:3A4A612D75ED088EA542ACAC52F9F45987488D1C&tr=udp://tracker.opentrackr.org:1337/announce"

### Waifu Diffusion

**v1.3**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:AWJJJZNFOOK7R2XXXGZ4GFNKUEU2TSFP&dn=wd-v1-3-float16.ckpt&xl=2132889245&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce"

**v1.2**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
%cd "{model_storage_dir}"
!aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:153590FD7E93EE11D8DB951451056C362E3A9150&dn=wd-v1-2-full-ema-pruned.ckpt&tr=udp://tracker.opentrackr.org:1337"

### trinart_stable_diffusion_v2

Another anime finetune. Designed to nudge SD to an anime/manga style. Seems to be more "stylized" and "artistic" than Waifu Diffusion, if that makes any sense.

The 60,000 steps version is the original, the 115,000 and 95,000 versions is the 60,000 with additional training. Use the 60,000 step version if the style nudging is too much.

[See the comparison here.](https://cyberes.github.io/stable-diffusion-models/#model-comparison)

**60000**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt -d "{model_storage_dir}" -o "trinart2_step60000.ckpt"

**95000**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt -d "{model_storage_dir}" -o "trinart2_step95000.ckpt"

**115000**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt -d "{model_storage_dir}" -o "trinart2_step115000.ckpt"

### trinart_characters_19-2m_stable_diffusion_v1

Trained on 19.2M anime/manga-style images. This model seeks for a sweet spot between artistic style and anatomical quality.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
!aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 https://huggingface.co/naclbit/trinart_characters_19.2m_stable_diffusion_v1/resolve/main/trinart_characters_it4_v1.ckpt -d "{model_storage_dir}" -o "trinart_characters_it4_v1.ckpt"

### NovelAI Leak

**animefull-final-pruned**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
metalink = 'magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce'
import re
infohash = re.search("^magnet:\?xt=urn:btih:(.*?)&.*?$", metalink).group(1)
import subprocess
tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
%cd "{tmp_dir}"
# Have to convert the metalink to a torrent file so aria2c can read the files inside
!aria2c -d . --bt-metadata-only=true --bt-save-metadata=true --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"
!aria2c --select-file=64,65 --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{infohash}.torrent"
!mv "novelaileak/stableckpt/animefull-final-pruned/config.yaml" "{model_storage_dir}/novelai-animefull-final-pruned.yaml"
!mv "novelaileak/stableckpt/animefull-final-pruned/model.ckpt" "{model_storage_dir}/novelai-animefull-final-pruned.ckpt"

**animesfw-final-pruned**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
metalink = 'magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce'
import re
infohash = re.search("^magnet:\?xt=urn:btih:(.*?)&.*?$", metalink).group(1)
import subprocess
tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
%cd "{tmp_dir}"
# Have to convert the metalink to a torrent file so aria2c can read the files inside
!aria2c -d . --bt-metadata-only=true --bt-save-metadata=true --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"
!aria2c --select-file=70,71 --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{infohash}.torrent"
!mv "novelaileak/stableckpt/animesfw-final-pruned/config.yaml" "{model_storage_dir}/novelai-animesfw-final-pruned.yaml"
!mv "novelaileak/stableckpt/animesfw-final-pruned/model.ckpt" "{model_storage_dir}/novelai-animesfw-final-pruned.ckpt"

**Hypernetworks**

A hypernetwork is trained much like a neural network and helps to guide the neural net towards the intended output.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

!apt update
!apt install -y aria2
metalink = 'magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce'
import re
infohash = re.search("^magnet:\?xt=urn:btih:(.*?)&.*?$", metalink).group(1)
import subprocess
tmp_dir = subprocess.check_output(['mktemp', '-d']).decode('ascii').strip('\n')
%cd "{tmp_dir}"
# Have to convert the metalink to a torrent file so aria2c can read the files inside
!aria2c -d . --bt-metadata-only=true --bt-save-metadata=true --bt-max-peers=120 --summary-interval=0 --file-allocation=none "magnet:?xt=urn:btih:5bde442da86265b670a3e5ea3163afad2c6f8ecc&dn=novelaileak&tr=udp%3A%2F%2Ftracker.opentrackr.org%3A1337%2Fannounce&tr=udp%3A%2F%2F9.rarbg.com%3A2810%2Fannounce&tr=udp%3A%2F%2Ftracker.openbittorrent.com%3A6969%2Fannounce&tr=http%3A%2F%2Ftracker.openbittorrent.com%3A80%2Fannounce&tr=udp%3A%2F%2Fopentracker.i2p.rocks%3A6969%2Fannounce"
!aria2c --select-file=76,81,82,83,84,85,86,87,88,89,90,91,92,93 --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{infohash}.torrent"
# -exec mv doesn't work with python variables so we'll set an environment variable instead
import os
os.environ["MODEL_STORAGE_DIR"] = model_storage_dir
!rm novelaileak/stableckpt/extra-sd-prune/sd-prune/anime700k-64bs-0.1ucg-penultimate-1epoch-clip-ema-continue-76000.pt # aria2 downloads this file even though I told it not to
!find novelaileak/ -type f -name '*.pt' -exec mv {} "$MODEL_STORAGE_DIR/hypernetworks" \;

## Clean up and restart the kernel

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir pip_cache_dir
    test = [model_storage_dir, repo_storage_dir, pip_cache_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

# Get some storage back

if not pip_cache_dir:
    !pip cache purge
    !echo "Purged pip cache"
!cd "{model_storage_dir}" && rm *.aria2
!apt remove --purge -y aria2 p7zip-full
!apt autoremove --purge -y
!apt clean

# Restart the kernel
import os
os.kill(os.getpid(), 9)

## Link the models directory

Create symlinks. The file will be stored in the models storage directory and linked to where the WebUI expects the files to be.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir link_novelai_anime_vae search_paperspace_datasets
    test = [model_storage_dir, repo_storage_dir, link_novelai_anime_vae, search_paperspace_datasets]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

import os
import glob
import subprocess
import re
from pathlib import Path
import sys

if not os.path.isdir(model_storage_dir):
    print('Your model storage directory does not exist:', model_storage_dir)
    sys.exit(1)

webui_model_path = Path(repo_storage_dir, 'stable-diffusion-webui/models')
webui_sd_models = Path(webui_model_path, 'Stable-diffusion')
webui_hypernetworks = Path(webui_model_path, 'hypernetworks')
webui_vaes = Path(webui_model_path, 'VAE')

def delete_broken_symlinks(dir):
    deleted = False
    for file in os.listdir(dir):
        path = f'{dir}/{file}'
        if os.path.islink(path) and not os.path.exists(os.readlink(path)):
            print(f'Symlink broken, removing: {file}')
            os.unlink(path)
            deleted = True
    if deleted:
        print('')
        
def create_symlink(source, dest):
    if os.path.isdir(dest):
        dest = Path(dest, os.path.basename(source))
    if not os.path.exists(dest):
        !ln -s "{source}" "{dest}"
    print(source, '->', Path(dest).absolute())

# Check for broken symlinks and remove them
print('Removing broken symlinks...')
delete_broken_symlinks(webui_sd_models)
delete_broken_symlinks(webui_hypernetworks)
delete_broken_symlinks(webui_vaes)

# Link .ckpt files (recursive)
print('\nLinking .ckpt files...')
for file in glob.glob(f'{model_storage_dir}/**/*.ckpt', recursive=True):
    if not Path(file).parent.parts[-1] in ['hypernetworks', 'vae'] :
        if not os.path.exists(file):
            print('New model:', os.path.basename(file))
        create_symlink(file, webui_sd_models)

# Link config yaml files
print('\nLinking config .yaml files...')
for file in glob.glob(f'{model_storage_dir}/**/*.yaml', recursive=True):
    create_symlink(file, webui_sd_models)

# Link hypernetworks
print('\nLinking hypernetworks...')
source_hypernetworks = Path(model_storage_dir, 'hypernetworks')
if os.path.isdir(source_hypernetworks):
    for file in os.listdir(source_hypernetworks):
        create_symlink(Path(source_hypernetworks, file), webui_hypernetworks)
else:
    print('Hypernetwork storage directory not found:', source_hypernetworks)

# Link VAEs
print('\nLinking VAEs...')
source_vaes = Path(model_storage_dir, 'vae')
if os.path.isdir(source_vaes):
    for file in os.listdir(source_vaes):
        create_symlink(Path(source_vaes, file), webui_vaes)
else:
    print('VAE storage directory not found:', source_vaes)

# Link the NovelAI files for each of the NovelAI models
print('\nLinking NovelAI files for each of the NovelAI models...')
for model in glob.glob(f'{model_storage_dir}/novelai-*.ckpt'):
    yaml = model.replace('.ckpt', '.yaml')
    if os.path.exists(yaml):
        # if not os.path.exists(yaml):
        print(f'New NovelAI model config: {yaml}')
        create_symlink(yaml, webui_sd_models)

if link_novelai_anime_vae:
    print('\nLinking NovelAI anime VAE...')
    for model in glob.glob('novelai-*.ckpt'):
        if os.path.exists(Path(model_storage_dir, 'hypernetworks/animevae.pt')):
            vae = model.replace('.ckpt', '.vae.pt')
            if not os.path.exists(webui_vaes):
                print(f'Linking NovelAI {vae} and {model}')
            create_symlink(Path(model_storage_dir, 'hypernetworks/animevae.pt'), webui_vaes)
        else:
            print(f'{model_storage_dir}/hypernetworks/animevae.pt NOT FOUND')
            

if search_paperspace_datasets:
    if os.path.isdir('/datasets'):
        print('\nSearching /datasets')
        for file in glob.glob(f'/datasets/**/*.ckpt', recursive=True):
            create_symlink(file, webui_sd_models)
        for file in glob.glob(f'/datasets/**/*.yaml', recursive=True):
            create_symlink(file, webui_sd_models)
    else:
        print('\nNo datasets mounted!')

# Launch the WebUI

Run this block to launch the WebUI. You will get a link to nnn.gradio.app, that's your WebUI. Follow it.

See [shared.py](https://github.com/AUTOMATIC1111/stable-diffusion-webui/blob/master/modules/shared.py#L22) to view the code for the launch args. There's a lot of good info in there about exactly what the args do. If you aren't a programmer, [here's the wiki](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Command-Line-Arguments-and-Settings).

#### Troubleshooting
- If you have any issues, try restarting the kernel.
- `EOFError: Ran out of input` probably means you ran out of storage space and the model `.ckpt` file wasn't downloaded completely. Try cleaning up your files. There are some helpful scripts in the Tools section below.
- `The file may be malicious, so the program is not going to read it` means the program encountered unexpected data in the model file (the technical term is "pickle"). Merging models can cause this. You can disable this feature by setting `disable_pickle_check` to True in the settings block.
- Try updating your notebook using the block in the Tools section below.
- If you're still having issues, delete `stable-diffusion-webui` and reinstall.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir activate_xformers activate_deepdanbooru activate_medvram disable_pickle_check gradio_port gradio_auth ui_theme insecure_extension_access
    test = [model_storage_dir, repo_storage_dir, activate_xformers, activate_deepdanbooru, activate_medvram, disable_pickle_check, gradio_port, gradio_auth, ui_theme, insecure_extension_access]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

%cd "{repo_storage_dir}/stable-diffusion-webui"

# Code to set the options you want as defined in the very first block
x_arg = '--xformers' if activate_xformers else ''
dd_arg = '--deepdanbooru' if activate_deepdanbooru else ''
mvram_arg = '--medvram' if activate_medvram else ''
pickled = '--disable-safe-unpickle' if disable_pickle_check else ''
port = f'--port {gradio_port}' if gradio_port else '--share'
auth = f'--gradio-auth {gradio_auth} --enable-insecure-extension-access' if gradio_auth else ''
theme = f'--theme {ui_theme}' if ui_theme else ''
insecure_extension_access = '--enable-insecure-extension-access' if insecure_extension_access else ''

# Launch args go below:
!python webui.py {x_arg} {dd_arg} {mvram_arg} {pickled} {port} {auth} {theme} --gradio-debug

# Export Generations

This block will rename and compress the outputs with 7zip max compression. It expects you to have `log/` and `outputs/` in `/notebooks/stable-diffusion-webui/`.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir export_storage_dir
    test = [model_storage_dir, repo_storage_dir, export_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

import os

if not os.path.exists(export_storage_dir):
    os.makedirs(export_storage_dir)

!if [ $(dpkg-query -W -f='${Status}' p7zip-full 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y p7zip-full; fi # install 7z if it isn't already installed
from datetime import datetime
datetime_str = datetime.now().strftime('%m-%d-%Y_%H:%M:%S')
%cd "{export_storage_dir}"
!mkdir -p "{datetime_str}/log"
!cd "{repo_storage_dir}/stable-diffusion-webui/log/" && mv * "{export_storage_dir}/{datetime_str}/log"
!cd "{repo_storage_dir}/stable-diffusion-webui/outputs/" && mv * "{export_storage_dir}/{datetime_str}"
!find "{export_storage_dir}/{datetime_str}" -name .ipynb_checkpoints -exec rm -rf {{}} +
!7z a -t7z -m0=lzma2 -mx=9 -mfb=64 -md=32m -ms=on "{datetime_str}.7z" "{export_storage_dir}/{datetime_str}/"

### Delete old output folder

This block will delete the folder you just compressed.

In [ ]:
# !rm -rf "{export_storage_dir}/{datetime_str}/"
# !echo Deleted {export_storage_dir}/{datetime_str}/

# Tools

### Show graphics card info

In [ ]:
!nvidia-smi

### Download the latest version of this notebook from Github

Run this and refresh the page (press F5). Don't save anything or you will overwrite the downloaded file.

In [ ]:
!mv /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb.backup # save your old notebook to a backup
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/StableDiffusionUI_Voldemort_paperspace.ipynb -O /notebooks/StableDiffusionUI_Voldemort_paperspace.ipynb
!echo "Downloaded! Now, refresh the page (press F5). Don't save anything or you will overwrite the downloaded file."

### Reset Repository

Sometimes AUTOMATIC1111 breaks something. Go to https://github.com/AUTOMATIC1111/stable-diffusion-webui/commits/master and choose a commit to revert to.

If you're looking for a specific date, do: `git log --since='Sept 17 2022' --until='Sept 18 2022'`


**This shouldn't delete your outputs or any changes you've made to files, but I'd back up anything important just to be safe.**

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)
%cd "{repo_storage_dir}/stable-diffusion-webui"
!git reset --hard <commit>

### Delete .ipynb_checkpoints

Jupyter stores temporary files in folders named `.ipynb_checkpoints`. It gets a little excessive sometimes so if you're running low on storage space or getting weird errors about a directory named `.ipynb_checkpoints`, run this block.

In [ ]:
!find . -type d -name .ipynb_checkpoints -delete

### Reset storage

This will delete ALL your files in `/notebooks/`, `/storage/`, `model_storage_dir`, and `repo_storage_dir`. Use if you're having issues with zero storage space and you don't want to delete your notebook.

In [ ]:
# Uncomment the lines below to run this block. You can highlight the lines and do ctrl + /
# %store -r model_storage_dir repo_storage_dir
# try:
#     test = [model_storage_dir, repo_storage_dir]
# except NameError as e:
#     print("There is an issue with your variables.")
#     print("Please go back to the first block and make sure your settings are correct, then run the cell.")
#     print('Error:', e)
#     import sys
#     sys.exit(1)
# !rm -rf /storage/*
# !mv /notebooks/*.ipynb / # move the notebook out of the directory before we nuke it
# !rm -rf /notebooks/*
# !mv /*.ipynb /notebooks/ # move it back
# !rm -rf {model_storage_dir}
# !rm -rf {repo_storage_dir}

### Build and Install Xformers

This is an advanced feature that should boost your generation speeds.

1. Run the block below to download the install script to `/notebooks/`
2. Go to https://developer.nvidia.com/cuda-gpus and find the Cuda arch for your GPU model. It's likely 7.5, but double check.
3. Once you have read these instructions, uncomment the second line and insert your Cuda arch.
4. Enable xformers in the settings block above.

If you have any issues, open the Jpyter Lab and run `build-xformers.sh` from the terminal.

In [ ]:
!wget https://raw.githubusercontent.com/Engineer-of-Stuff/stable-diffusion-paperspace/main/other/build-xformers.sh -O /notebooks/build-xformers.sh
# !bash /notebooks/build-xformers.sh [your cuda arch]
!echo "COMPLETED!"

### Automated Model Downloader

Here's a tool to download a model from a torrent magnet link, web link, Google Drive, or HuggingFace.

In [ ]:
try:
    %store -r model_storage_dir repo_storage_dir
    test = [model_storage_dir, repo_storage_dir]
except NameError as e:
    print("There is an issue with your variables.")
    print("Please go back to the first block and make sure your settings are correct, then run the cell.")
    print('Error:', e)
    import sys
    sys.exit(1)

model_uri = input('URI of model to download: ')
import re
import requests
import gdown
   
def dl_web_file(web_dl_file):
    %cd "{model_storage_dir}"
    # We're going to use aria2 to split the download into threads which will allow us to download
    # the file very fast even if the site serves the file slow.
    !if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
    !aria2c --file-allocation=none -c -x 16 -s 16 --summary-interval=0 "{web_dl_file}" 

magnet_match = re.search(r'magnet:\?xt=urn:btih:[\-_A-Za-z0-9&=%.]*', model_uri)
web_match = re.search(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})', model_uri)

if magnet_match:
    !if [ $(dpkg-query -W -f='${Status}' aria2 2>/dev/null | grep -c "ok installed") = 0 ]; then sudo apt update && sudo apt install -y aria2; fi
    %cd "{model_storage_dir}"
    bash_var = magnet_match[0]
    !aria2c --seed-time=0 --max-overall-upload-limit=1K --bt-max-peers=120 --summary-interval=0 --file-allocation=none "{bash_var}"
elif 'https://huggingface.co/' in model_uri:
    response = requests.head(web_match[0], allow_redirects=True)
    if 'octet-stream' not in response.headers['content-type']:
        response = requests.head(web_match[0].replace('/blob/', '/resolve/'), allow_redirects=True)
        if 'octet-stream' not in response.headers['content-type']:
            print(f'Wrong content-type: {response.headers["content-type"].split(";")[0]}')
            # clean exit here
        else:
            dl_web_file(web_match[0].replace('/blob/', '/resolve/'))
    else:
        dl_web_file(web_match[0])
elif 'https://drive.google.com' in model_uri:
    gdrive_file_id, _ = gdown.parse_url.parse_url(web_match[0])
    %cd "{model_storage_dir}"
    gdown.download(f"https://drive.google.com/uc?id={gdrive_file_id}&confirm=t")    
elif web_match:
    response = requests.head(web_match[0], allow_redirects=True)
    if 'octet-stream' not in response.headers['content-type']:
        print(f'Wrong content-type: {response.headers["content-type"].split(";")[0]}')
        # clean exit here
    else:
        dl_web_file(web_match[0])
else:
    print('Could not parse your URI.')
    # clean exit here